<font size=6><b>스토캐스틱 오실레이터(Stochastic Oscillator) - Backtest

# BackTrader
* ref : https://github.com/mementum/backtrader
* ref : https://www.backtrader.com/recipes/indicators/stochastic/stochastic/

In [1]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
import backtrader as bt

In [2]:
class Stochastic_Generic(bt.Indicator):
    '''
    This generic indicator doesn't assume the data feed has the components
    ``high``, ``low`` and ``close``. It needs three data sources passed to it,
    which whill considered in that order. (following the OHLC standard naming)
    '''
    lines = ('k', 'd', 'dslow',)
    params = dict(
        pk=5,
        pd=3,
        pdslow=3,
        movav=bt.ind.SMA,
        slowav=None,
    )

    def __init__(self):

        highest = bt.ind.Highest(self.High, period=self.p.pk)
        lowest  = bt.ind.Lowest(self.Low  , period=self.p.pk)

        # raw K
        kraw = 100.0 * (self.Close - lowest) / (highest - lowest)

        # The standard k in the indicator is a smoothed versin of K
        self.l.k = k = self.p.movav(kraw, period=self.p.pd)

        # Smooth k => d
        slowav = self.p.slowav or self.p.movav  # chose slowav
        self.l.d = slowav(k, period=self.p.pdslow)

# 활용 예

In [3]:
import pandas as pd
import datetime as dt
from pykrx import stock
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='Malgun Gothic')
sns.set()

#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

## OHLCV data load

In [4]:
start  = "2021-06-01"
end    = "2021-12-31"
ticker = "005930"
# ---------------------------------------
df = stock.get_market_ohlcv(start, end, ticker)
#df = stock.get_market_ohlcv("2021-01-01", "2021-12-31", "005930")
df.columns = ["Open","High","Low","Close","Volume"]
df.head()

,Open,High,Low,Close,Volume
날짜,,,,,
2021-06-01,80500,81300,80100,80600,14058401
2021-06-02,80400,81400,80300,80800,16414644
2021-06-03,81300,83000,81100,82800,29546007
2021-06-04,82700,82700,81500,82200,18112259
2021-06-07,82700,82800,81600,81900,16496197


## SMA 기반 백테스팅

In [29]:
from datetime import datetime
import backtrader as bt
import locale

locale.setlocale(locale.LC_ALL, 'ko_KR')

# Create a subclass of Strategy to define the indicators and logic
class SmaCross(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=5,  # period for the fast moving average
        pslow=30  # period for the slow moving average
    )

    def __init__(self):    
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

        self.holding = 0

    def next(self):
        current_stock_price = self.data.close[0]

        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                available_stocks = self.broker.getcash() / current_stock_price
                self.buy(size=1)

        elif self.crossover < 0:  # in the market & cross to the downside
            self.close()  # close long position

    def notify_order(self, order):
        if order.status not in [order.Completed]:
            return

        if order.isbuy():
            action = 'Buy'
        elif order.issell():
            action = 'Sell'

        stock_price = self.data.close[0]
        cash = self.broker.getcash()
        value = self.broker.getvalue()
        self.holding += order.size

        print('%s[%d] holding[%d] price[%d] cash[%.2f] value[%.2f]'
              % (action, abs(order.size), self.holding, stock_price, cash, value))

cerebro = bt.Cerebro()  # create a "Cerebro" engine instance
cerebro.broker.setcash(100000)
cerebro.broker.setcommission(0.002)


data = bt.feeds.PandasData(dataname=df)
cerebro.adddata(data)  # Add the data feed

cerebro.addstrategy(SmaCross)  # Add the trading strategy

start_value = cerebro.broker.getvalue()
cerebro.run()  # run it all
final_value = cerebro.broker.getvalue()

print('* start value : %s won' % locale.format_string('%d', start_value, grouping=True))
print('* final value : %s won' % locale.format_string('%d', final_value, grouping=True))
print('* earning rate : %.2f %%' % ((final_value - start_value) / start_value * 100.0))

# cerebro.plot(style='candlestick',barup='red',bardown='blue',xtight=True,ytight=True, grid=True)

Buy[1] holding[1] price[82100] cash[16533.40] value[98633.40]
Sell[1] holding[0] price[74400] cash[92181.80] value[92181.80]
Buy[1] holding[1] price[77400] cash[14526.80] value[91926.80]
Sell[1] holding[0] price[72200] cash[87380.80] value[87380.80]
Buy[1] holding[1] price[70700] cash[15938.20] value[86638.20]
* start value : 100,000 won
* final value : 94,238 won
* earning rate : -5.76 %


## Stochastic 기반 벡테스팅

In [30]:
from datetime import datetime
import backtrader as bt
import locale

locale.setlocale(locale.LC_ALL, 'ko_KR')

# Create a subclass of Strategy to define the indicators and logic
class Stochastic_Generic(bt.Strategy):
    params = dict(
        pk=5,
        pd=3,
        pdslow=3,
        movav=bt.ind.SMA,
        slowav=None,
    )
    
    def __init__(self):
        print("---------------",self.data.close[0])#0 
        print("---------------",self.data.open[0])#4
        
        highest = bt.ind.Highest(self.data.high, period=self.p.pk)
        lowest  = bt.ind.Lowest(self.data.low,   period=self.p.pk)
        kraw = 100.0 * (self.data.close - lowest) / (highest - lowest)
        
        # The standard k in the indicator is a smoothed versin of K
        self.l.k = k = self.p.movav(kraw, period=self.p.pd)

        # Smooth k => d
        slowav = self.p.slowav or self.p.movav  # chose slowav
        self.l.d = slowav(k, period=self.p.pdslow)
        
        self.holding = 0
        
        
    def next(self):
        current_stock_price = self.data.close[0]
        
        if self.l.k > self.l.d :
            self.buy(size=1)
        elif self.l.k < self.l.d :
            self.sell(size=1)
    
    def notify_order(self, order):
        if order.status not in [order.Completed]:
            return

        if order.isbuy():
            action = 'Buy'
        elif order.issell():
            action = 'Sell'

        stock_price = self.data.close[0]
        cash = self.broker.getcash()
        value = self.broker.getvalue()
        self.holding += order.size

        print('%s[%d] holding[%d] price[%d] cash[%.2f] value[%.2f]'
              % (action, abs(order.size), self.holding, stock_price, cash, value))


cerebro = bt.Cerebro()  # create a "Cerebro" engine instance
cerebro.broker.setcash(100000)
cerebro.broker.setcommission(0.002)


data = bt.feeds.PandasData(dataname=df)
cerebro.adddata(data)  # Add the data feed

cerebro.addstrategy(Stochastic_Generic)  # Add the trading strategy

start_value = cerebro.broker.getvalue()
cerebro.run()  # run it all
final_value = cerebro.broker.getvalue()

print('* start value : %s won' % locale.format_string('%d', start_value, grouping=True))
print('* final value : %s won' % locale.format_string('%d', final_value, grouping=True))
print('* earning rate : %.2f %%' % ((final_value - start_value) / start_value * 100.0))

# cerebro.plot(style='candlestick',barup='red',bardown='blue',xtight=True,ytight=True, grid=True)

--------------- 78300.0
--------------- 78900.0
Sell[1] holding[-1] price[80500] cash[180638.40] value[100138.40]
Sell[1] holding[-2] price[80900] cash[261376.60] value[99576.60]
Buy[1] holding[-1] price[81800] cash[179713.60] value[97913.60]
Buy[1] holding[0] price[80900] cash[98451.40] value[98451.40]
Buy[1] holding[1] price[80500] cash[17189.20] value[97689.20]
Sell[1] holding[0] price[79900] cash[96729.80] value[96729.80]
Sell[1] holding[-1] price[80000] cash[176769.40] value[96769.40]
Sell[1] holding[-2] price[80100] cash[257108.40] value[96908.40]
Buy[1] holding[-1] price[81200] cash[176547.60] value[95347.60]
Buy[1] holding[0] price[81600] cash[94884.60] value[94884.60]
Buy[1] holding[1] price[81900] cash[13021.20] value[94921.20]
Sell[1] holding[0] price[80700] cash[93959.00] value[93959.00]
Sell[1] holding[-1] price[80100] cash[174298.00] value[94198.00]
Sell[1] holding[-2] price[80000] cash[254138.00] value[94138.00]
Sell[1] holding[-3] price[80400] cash[334077.80] value[9287